In [1]:
# Mostly adapted from # 03 - pick.ipynb

import numpy as np
from pydrake.all import (StartMeshcat, MeshcatVisualizer, DiagramBuilder,
                        Simulator, LeafSystem, RigidTransform, RotationMatrix,
                        PiecewisePose, TrajectorySource, Integrator, JacobianWrtVariable,
                        ConstantVectorSource, MeshcatVisualizerParams, Role)
from manipulation import FindResource
from manipulation.scenarios import MakeManipulationStation

# For diagram viz
from IPython.display import SVG
import pydot

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
## Question
# Why does the pseudo-inverse controller have its own plant, iiwa, and context?
# Seems like just a node for determining commands real-time

## Question
# Why does the pseudo inverse controller set the positions of the Kuka each time it reads 
# the measured positions of the kuka?

## !! Question
# We are compensating velocity, but what we really want to be controlling is position. If the differentiation
# has some error, we will never observe that our position is incorrect - we'll just continue to compensate for
# whatever our velocity time series is. How do we solve this issue? Is this an issue we need to solve?

## Question
# How does the pseudo-inverse controller call CalcOutput? I see it's attached to output port, but 
# where do "context" and "output" get passed through?

class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa_position", 7)
        self.DeclareVectorOutputPort("iiwa_velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G)
        output.SetFromVector(v)

In [4]:
class IIWA:
    def __init__(self, q0, traj):
        
        # Setup diagram builder components
        builder = DiagramBuilder()
        self.station = MakeManipulationStation(filename=FindResource("/home/jared/Desktop/RoboticRecycling/models/recycling.dmd.yaml"),
            package_xmls=["./package.xml"])
        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat)  
        self.collision = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat,
            MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))

        # In case of initializing empty station
        if traj is not None:
            # Connect trajectories to pseudo-inverse controller
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            V_G_source.set_name("V_WG")
            self.controller = builder.AddSystem(PseudoInverseController(self.plant))
            self.controller.set_name("PseudoInverseController")
            builder.Connect(V_G_source.get_output_port(), self.controller.GetInputPort("V_WG"))

            # Integrate controller velocity commands to get joint angles
            self.integrator = builder.AddSystem(Integrator(7))
            self.integrator.set_name("integrator")
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(self.station.GetOutputPort("iiwa_position_measured"),
                            self.controller.GetInputPort("iiwa_position"))
            
        # Fix station input ports for gripper controller to no motion
        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(wsg_position.get_output_port(),
                        self.station.GetInputPort("wsg_position"))
        
        # Build
        self.diagram = builder.Build()
        
        # House frames
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()
        
        # Develop context
        self.context = self.diagram.CreateDefaultContext()
        station_context = self.diagram.GetMutableSubsystemContext(self.station, self.context)
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, self.context)
        
        # Set initial joint positions
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])  
        
        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(self.context), q0)
            
        self.diagram.Publish(self.context)
        
        
    def Get_X_WG(self):
        
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, self.context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG
    
    
    def Simulate(self, t):
        
        self.simulator = Simulator(self.diagram, self.context)
        self.simulator.set_target_realtime_rate(1.0)
        self.visualizer.StartRecording()
        self.simulator.AdvanceTo(t)
        self.visualizer.PublishRecording()
        
        
    def ShowDiagram(self):
        
        SVG(pydot.graph_from_dot_data(self.diagram.GetGraphvizString())[0].create_svg())
        

In [5]:
meshcat.Delete()

## Question
# When starting with q0 = zeros and commanding some arbitrary position, the sim breaks because the dynamics
# encounter a singularity that makes some desired joint torque / velocity infinity. This controller doesn't
# account for singularities? What's stopping us from reaching a singularity in regular use? How to remedy?
# q0 = np.zeros(7)

q0 = np.array([ -1.57,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])

temp_iiwa = IIWA(q0, None)
X_init = temp_iiwa.Get_X_WG()

print(X_init)

# X_21 = RigidTransform(RotationMatrix().MakeZRotation(np.pi/12), [0,0,0])
# poses = [X_init]
# for i in range(12):
#     poses.append(X_21 @ poses[i])
# times = np.linspace(0, 10, num=len(poses))
# traj = PiecewisePose.MakeLinear(times, poses)

R = RotationMatrix(np.array([[0, 0, -1],
                             [1, 0, 0],
                             [0, -1, 0]]))
X_recycle = RigidTransform(R, [0.5, -0.2, 0.75])
X_waste = RigidTransform(R, [0.5, 0, 0.75])
X_organic = RigidTransform(R, [0.5, 0.2, 0.75])

traj = PiecewisePose.MakeLinear([0, 3, 5], [X_init, X_waste, X_waste])

iiwa = IIWA(q0, traj)

RigidTransform(
  R=RotationMatrix([
    [0.9999996921089161, 2.593293499167941e-05, -0.000784289205192317],
    [0.0007846004500206739, -0.05032888210082877, 0.9987323905975092],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[0.00035597561337895496, -0.4686661801810802, 0.6012858601360346],
)


In [59]:
iiwa.Simulate(10)